## קוד מבוא

#### ספריות

In [11]:
import os
import sys
import pathlib
import pandas as pd
import geopandas as gpd

In [12]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### העלאת משתנים להרצת הקוד

In [13]:
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

future_folder = os.path.dirname(software_data_folder_location)

create_forecast_basic_software_location = os.path.dirname(future_folder)

sys.path.append(create_forecast_basic_software_location)

create_forecast_basic_software_df = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(create_forecast_basic_software_location))

TAZ_V4_date = create_forecast_basic_software_df['location'][1]

## פונקציות

### פונקציות גלובליות

In [14]:
from global_functions import make_point, up_load_gdb, up_load_shp, up_load_df, logic_test_for_forecast, find_files_with_pattern, drop_geo

##  העלאת טבלאות אוכלוסייה

In [15]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

matching_files=find_files_with_pattern(r'{}\Intermediates\change_pop'.format(software_data_folder_location), 'pop_2025_iplan_changed')
file = pathlib.Path(matching_files[0])
v_date = file.name[:6]

year=['2025','2030','2035','2040','2045','2050']

for y in year: 
    col=['Taz_num',
 'aprt',
 'pop',
 'pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']
    
    folder_path=r'{}\Intermediates\change_pop'.format(software_data_folder_location)

    file_name='{}_pop_{}_iplan_changed'.format(v_date,y)

    locals()['forecast_pop_{}_iplan'.format(y)]=up_load_df(folder_path,file_name)[col]

## העלאת טבלת מצב קיים

In [16]:
matching_files=find_files_with_pattern(r'{}'.format(forecast_version_folder_location), '2020_jtmt_forcast_full')
file = pathlib.Path(matching_files[0])
file_name = file.name

col=['Taz_num',
     'Taz_name',
     'Muni_Heb',
     'jeru_metro',
     'zonetype',
     'in_jerusal',
     'main_secto',
     'aprt_20',
     'student_dorms',
     'student_yeshiva_with_add_from_old',
     'student',
     'uni_students','emp_from_uni_student','pop','emp_from_Yeshiva_student','yeshiva_dorms_pop_15',
     'yeshiva_dorms_pop_20',
     'yeshiva_dorms_pop_25',
     'yeshiva_dorms_pop_sum',]

new_column_names = {'aprt_20': 'aprt','student_yeshiva_with_add_from_old':'student_yeshiva'}

forecast_2020=up_load_df(forecast_version_folder_location,file_name[:-5])[col]
forecast_2020=forecast_2020.rename(columns=new_column_names)

In [17]:
forecast_2020['jew']=0

forecast_2020.loc[(forecast_2020['main_secto']=="U_Orthodox")|(forecast_2020['main_secto']=="Jewish"),'jew']=1

In [18]:
folder_path=r'{}\background_files'.format(software_data_folder_location)

taz_sector_change=up_load_df(folder_path,'230704_taz_num_convert_from_jewish_to_u_orthodox')

lst_change_sector=list(taz_sector_change.Taz_num)

forecast_2020.loc[(forecast_2020['Taz_num'].isin(lst_change_sector))&(forecast_2020['zonetype']=='Judea and Samaria'),'main_secto']='U_Orthodox'

## העלאת שכבת לא עוקב

In [19]:
EMP_kibolet=up_load_gdb(r'{}\background_files\EMP_KIBOLET.gdb'.format(software_data_folder_location),'EMP_kibolet')

EMP_kibolet=EMP_kibolet.fillna(0)

EMP_kibolet['emp_kayim_no_palestinians']=EMP_kibolet['kayim_emp']-EMP_kibolet['Palestinians']

EMP_kibolet['emp_current']=EMP_kibolet['emp_kayim_no_palestinians']

In [20]:
col=[ 'F2025',
 'F2030',
 'F2035',
 'F2040',
 'F2045',
 'F2050']

EMP_kibolet[col]=EMP_kibolet[col].apply(lambda x: x * (EMP_kibolet['kibolt']-EMP_kibolet['kayim_emp']))

EMP_kibolet[col]=EMP_kibolet[col].cumsum(axis=1)

EMP_kibolet[col]=EMP_kibolet[col].apply(lambda x: x +EMP_kibolet['emp_kayim_no_palestinians'])

emp_not_okev_current=EMP_kibolet['emp_kayim_no_palestinians'].sum().item()

## אזורי תעסוקה עיבודים והצלבה עם אזורי תנועה

##### העלאת שכבה

In [21]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_{}_with_geo_info.shp'.format(create_forecast_basic_software_location, TAZ_V4_date))

In [12]:
taz['taz_area']=taz.area

EMP_kibolet['emp_area']=EMP_kibolet.area

EMP_kibolet['ID']=EMP_kibolet.index

In [13]:
EMP_kibolet_by_taz=gpd.overlay(taz[['Taz_num','taz_area','geometry']],EMP_kibolet[['ID','emp_area','geometry']])

EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

EMP_kibolet_by_taz['pre_small_area_emp']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['emp_area']

EMP_kibolet_by_taz['pre_small_area_taz']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['taz_area']

EMP_kibolet_by_taz=EMP_kibolet_by_taz.query('(pre_small_area_taz >0.7) | (pre_small_area_emp >0.1)')[['ID','Taz_num','geometry']]

EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

EMP_kibolet_by_taz=EMP_kibolet_by_taz.set_index('ID')

EMP_kibolet_by_taz['id_area_for_pre']=drop_geo(EMP_kibolet_by_taz).groupby(by='ID').sum()['small_area']

EMP_kibolet_by_taz['prec_from_id']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['id_area_for_pre']

C:\Users\dpere\AppData\Local\Temp\ipykernel_6232\1124101454.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 279 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  EMP_kibolet_by_taz=gpd.overlay(taz[['Taz_num','taz_area','geometry']],EMP_kibolet[['ID','emp_area','geometry']])


In [14]:
EMP_kibolet['geometry_buff']=EMP_kibolet.buffer(250)

##### העלאת טבלאות

In [15]:
city_muni=['מודיעין עילית','בית שמש','ירושלים','מודיעין - מכבים - רעות']

okev_factor=up_load_df(r'{}\background_files'.format(software_data_folder_location),'okev_factor').set_index('sector')

work_factor=up_load_df(r'{}\background_files'.format(software_data_folder_location),'work_factor_230719').set_index('sector')

## התפלגות אזורים התפלגות ענפי תעסוקה

##### העלאת שכבה

In [16]:
emp_category_type=up_load_shp(r'{}\background_files\emp_category_type.shp'.format(software_data_folder_location)).fillna(0)

In [17]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

emp_category_type[col]=emp_category_type[col]/100

emp_category_type=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],emp_category_type).set_index('Taz_num')

## הכנה ללופ

##### העלאת שכבה

In [18]:
commuting=up_load_shp(r'{}\background_files\commuting_230712.shp'.format(software_data_folder_location))

In [19]:
pre_woman=0.5

pre_man=1-pre_woman

work_age=[ 'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60']

under_work_age=[ 'pop_15', 'pop_20']

over_work_age=[ 'pop_65', 'pop_70', 'pop_75up']

In [20]:
 #אלו השדות שאולי אני יוריד כאן נכנס המצב הנוכחי
col=['Taz_num',
     'Taz_name',
     'Muni_Heb',
     'jeru_metro',
     'zonetype',
     'in_jerusal',
     'main_secto','jew',
     'student_dorms',
     'student_yeshiva',
     'student',
     'uni_students','emp_from_uni_student','emp_from_Yeshiva_student','yeshiva_dorms_pop_15',
     'yeshiva_dorms_pop_20',
     'yeshiva_dorms_pop_25',
     'yeshiva_dorms_pop_sum',]

forecast_current=forecast_2020[col]

## לופ

In [21]:
year=['2025','2030','2035','2040','2045','2050']

for y in year:
    # חיבור מאפייני אזורי התנועה משכבת אזורי תנועה מצב קיים
    forecast=pd.merge(forecast_current,locals()['forecast_pop_{}_iplan'.format(y)],on="Taz_num",how='right')
    
    forecast['student_demand']=forecast['pop_0']/5*3*0.5+forecast['pop_0']/5*2+forecast['pop_5']+forecast['pop_10']+forecast['pop_15']/5*3+forecast['pop_0']/5*3*0.5

    demad_for_students_by_muni_and_sector=forecast.pivot_table(index=('Muni_Heb','jew'),aggfunc=sum)[['student_demand','student']]

    demad_for_students_by_muni_and_sector['add_students_to_kayim_from_demand']=demad_for_students_by_muni_and_sector['student_demand']-demad_for_students_by_muni_and_sector['student']
    demad_for_students_by_muni_and_sector.loc[demad_for_students_by_muni_and_sector['add_students_to_kayim_from_demand']<0,'add_students_to_kayim_from_demand']=0

    demad_for_students_by_muni_and_sector=demad_for_students_by_muni_and_sector.reset_index()

    col=['Muni_Heb','jew','add_students_to_kayim_from_demand']

    forecast=forecast.merge(demad_for_students_by_muni_and_sector[col],on=['Muni_Heb','jew'])

    forecast['student_to_fill_demand']=forecast['student_demand']-forecast['student']

    forecast.loc[forecast['student_to_fill_demand']<0,'student_to_fill_demand']=0

    forecast=forecast.merge(forecast.groupby(by=['Muni_Heb','jew']).sum()[['student_to_fill_demand']].reset_index(),on=['Muni_Heb','jew'],suffixes=('','_sum'))

    forecast['student_to_fill_demand_pre']=forecast['student_to_fill_demand']/forecast['student_to_fill_demand_sum']

    forecast['add_to_student_current']=forecast['student_to_fill_demand_pre']*forecast['add_students_to_kayim_from_demand']
        
    forecast=forecast.fillna(0)
    
    forecast['student']=forecast['student']+forecast['add_to_student_current']
        
    ## תעסוקה בעקבות תלמידים 
    emp_education_per_student=7
    
    forecast['emp_from_student']=forecast['student']/emp_education_per_student

    ## סטודנטים 
    forecast['uni_students_pre']=forecast['uni_students']/forecast['uni_students'].sum().item()

    uni_students_pre_from_age_20_30=0.3

    uni_students=forecast.query('main_secto!="Palestinian"')[['pop_20','pop_25']].sum().sum()*uni_students_pre_from_age_20_30
    
    dorms_vs_uni_students=(forecast['student_dorms'].sum().item())/(forecast['uni_students'].sum().item())
        
    forecast['add_uni_students']=forecast['uni_students_pre']*uni_students-forecast['uni_students']
    
    forecast['uni_students']=forecast['uni_students']+forecast['add_uni_students']
        
    growth_dorms=((forecast['uni_students'].sum().item())*dorms_vs_uni_students)/(forecast['student_dorms'].sum().item())
        
    forecast['student_dorms']=forecast['student_dorms']*growth_dorms
    
    ## תעסוקה בעקבות סטודנטים 
    emp_Education_per_uni_student=0.18
    
    forecast['emp_from_uni_student']=forecast['add_uni_students']*emp_Education_per_uni_student+forecast['emp_from_uni_student']   
 
    ## תלמידי ישיבה 
    student_yeshiva_pre_from_age_20_30=0.52

    total_student_yeshiva=forecast.query('jew==1')[['pop_20','pop_25']].sum().sum()*student_yeshiva_pre_from_age_20_30

    forecast['student_yeshiva_pre']=forecast['student_yeshiva']/forecast['student_yeshiva'].sum().item()
    
    ## תעסוקה בעקבות ישיבה 
    forecast['student_yeshiva_vs_yeshiva_emp']=forecast['student_yeshiva']/forecast['emp_from_Yeshiva_student']
    
    forecast['student_yeshiva_growth']=(forecast['student_yeshiva_pre']*total_student_yeshiva)/forecast['student_yeshiva']
    
    forecast['student_yeshiva']=forecast['student_yeshiva_pre']*total_student_yeshiva
    
    forecast['emp_from_Yeshiva_student']=forecast['student_yeshiva']/forecast['student_yeshiva_vs_yeshiva_emp']
   
    ##תוספת בפנימיות בישיבות
    forecast['yeshiva_dorms_pop_15']=forecast['yeshiva_dorms_pop_15']*forecast['student_yeshiva_growth']
    forecast['yeshiva_dorms_pop_20']=forecast['yeshiva_dorms_pop_20']*forecast['student_yeshiva_growth']
    forecast['yeshiva_dorms_pop_25']=forecast['yeshiva_dorms_pop_25']*forecast['student_yeshiva_growth']
    col_dorms_yeshiva=['yeshiva_dorms_pop_15', 'yeshiva_dorms_pop_20', 'yeshiva_dorms_pop_25']
    forecast['yeshiva_dorms_pop_sum']=forecast[col_dorms_yeshiva].sum(axis=1)
    
    ## כוללים 
    forecast['kollim_demand']=(forecast['pop_20']*0.8+forecast['pop_25']*0.65+forecast['pop_30']*0.30+forecast['pop_35']*0.30+forecast['pop_40']*0.30+forecast['pop_45']*0.20+forecast['pop_50']*0.20+forecast['pop_55']*0.20+forecast['pop_60']*0.20)*0.5 #הכפלה בחצי בשביל לקבל אוכלוסיית גברים מעורכת

    forecast.loc[forecast['main_secto']!='U_Orthodox','kollim_demand']=0  #אל אף שאנחנו יודעים שיש כוללים באזורים שהם לא מוגדרים כחרדים

    forecast['add_from_kollim_demand']=0

    forecast.loc[(forecast['main_secto']=='U_Orthodox')&(forecast['kollim_demand']>forecast['student_yeshiva']),'add_from_kollim_demand']=forecast['kollim_demand']-forecast['student_yeshiva']

    forecast['student_yeshiva_and_kollim']=forecast['add_from_kollim_demand']+forecast['student_yeshiva']
    
    forecast=forecast.fillna(0)

    ## תעסוקה סך חינוך 
    forecast['emp_Education']=forecast['emp_from_student']+forecast['emp_from_Yeshiva_student']+forecast['emp_from_uni_student']
    
    emp_Education=forecast.query('main_secto!="Palestinian"')['emp_Education'].sum().item()

    #אזורי תנועה שלא צריכים עוקב    
    EMP_buffer=EMP_kibolet.set_geometry('geometry_buff').query('F{} > 0'.format(y))

    EMP_buffer=EMP_buffer.dissolve()[['geometry_buff']]

    emp_buffer_taz=gpd.overlay(EMP_buffer,taz)

    emp_buffer_taz['emp_samll_area']=emp_buffer_taz.area

    emp_buffer_taz['emp_pre_from_taz']=emp_buffer_taz['emp_samll_area']/emp_buffer_taz['taz_area']

    taz_num_no_need_okev=emp_buffer_taz.loc[emp_buffer_taz['emp_pre_from_taz']>0.4].Taz_num.to_list()
       
    forecast['okev']=0
    
    forecast.loc[(~forecast['Taz_num'].isin(taz_num_no_need_okev))&(taz['main_secto']!="Palestinia"),'okev']=1

    forecast.loc[forecast['main_secto']=="arabs_behined_seperation_wall",'okev']=1
    
    ## יצירת תעסוקה כנגזר מעוקב
    forecast['emp_okev']=0

    forecast.loc[(forecast['jew']==0)&(forecast['okev']==1),'emp_okev']=forecast['aprt']*okev_factor.loc['arab','city']

    forecast.loc[(forecast['main_secto']=="U_Orthodox")&(~forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*okev_factor.loc['U_Orthodox','sub']

    forecast.loc[(forecast['main_secto']=="U_Orthodox")&(forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*okev_factor.loc['U_Orthodox','city']

    forecast.loc[(forecast['main_secto']=="Jewish")&(forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*okev_factor.loc['Jewish','city']

    forecast.loc[(forecast['main_secto']=="Jewish")&(~forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*okev_factor.loc['Jewish','sub']

    emp_okev=forecast['emp_okev'].sum().item()
        
    ##  הוספת סטודנטים  לאוכלוסיה ולהתפלגות גילים והוספת תלמידי ישיבה (באזורים חרדיים)
    ## הוספת סטודנטים לאוכלוסיה ולהתפלגות גילים
    forecast=forecast.fillna(0)

    forecast['pop_without_dorms_yeshiva']=forecast['pop']

    forecast['pop']=forecast['pop']+forecast['student_dorms']

    forecast['pop_20_just_from_aprt']=forecast['pop_20']

    forecast['pop_25_just_from_aprt']=forecast['pop_25']

    forecast['pop_20']=forecast['pop_20']+forecast['student_dorms']*0.6

    forecast['pop_25']=forecast['pop_25']+forecast['student_dorms']*0.4

    ## הוספת תלמידי ישיבה (באזורים חרדיים) אל כמות אוכלוסיה,התפלגות גילים 
    forecast['pop_15_just_from_aprt']=forecast['pop_15']
    
    forecast.loc[forecast['main_secto']=="U_Orthodox",'pop']=forecast['pop']+forecast['yeshiva_dorms_pop_sum']

    forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_15']=forecast['pop_15']+forecast['yeshiva_dorms_pop_15']

    forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_20']=forecast['pop_20']+forecast['yeshiva_dorms_pop_20']
    
    forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_25']=forecast['pop_25']+forecast['yeshiva_dorms_pop_25']

    ## הוספת סטודנטים אל כמות משקי בית
    forecast=forecast.fillna(0)

    forecast['hh']=forecast['aprt']+forecast['student_dorms']/1.5

    forecast['hh']=forecast['hh']+forecast['yeshiva_dorms_pop_sum']
   
    ## יצירת מסגרת מועסקים למרחב
    forecast['work_age']=forecast[work_age].sum(axis=1)

    forecast['under_work_age']=forecast[under_work_age].sum(axis=1)

    forecast['over_work_age']=forecast[over_work_age].sum(axis=1)

    sector=['U_Orthodox','Jewish','Arab','arabs_behined_seperation_wall']
    
    for s in sector:
        
        work_age_factor_woman=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='work_age') & (work_factor['gender']=='woman')& (work_factor['year']==int(y)),'value'].item()
        work_age_factor_man=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='work_age') & (work_factor['gender']=='man')& (work_factor['year']==int(y)),'value'].item()
        under_work_age_factor_woman=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='under_work_age') & (work_factor['gender']=='woman')& (work_factor['year']==int(y)),'value'].item()
        under_work_age_factor_man=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='under_work_age') & (work_factor['gender']=='man')& (work_factor['year']==int(y)),'value'].item()
        over_work_age_factor_woman=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='over_work_age') & (work_factor['gender']=='woman')& (work_factor['year']==int(y)),'value'].item()
        over_work_age_factor_man=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='over_work_age') & (work_factor['gender']=='man')& (work_factor['year']==int(y)),'value'].item()
        
        forecast.loc[forecast['main_secto']==s,'pop_emp']=forecast['work_age']*pre_woman*work_age_factor_woman+forecast['work_age']*pre_man*work_age_factor_man
        forecast.loc[forecast['main_secto']==s,'pop_emp']=forecast['pop_emp']+forecast['under_work_age']*pre_woman*under_work_age_factor_woman+forecast['under_work_age']*pre_man*under_work_age_factor_man
        forecast.loc[forecast['main_secto']==s,'pop_emp']=forecast['pop_emp']+forecast['over_work_age']*pre_woman*over_work_age_factor_woman+forecast['over_work_age']*pre_man*over_work_age_factor_man
     
    ## חישוב אבטלה
    forecast['pop_emp_employed']=0

    forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_emp_employed']=forecast['pop_emp']*0.96

    forecast.loc[forecast['main_secto']=="Jewish",'pop_emp_employed']=forecast['pop_emp']*0.97

    forecast.loc[forecast['jew']==0,'pop_emp_employed']=forecast['pop_emp']*0.96

    ## חישוב יוממות
    forecast=forecast.set_index('Taz_num')

    forecast['commuting']=gpd.sjoin(make_point(taz)[['Taz_num','centroid']],commuting[['commuting', 'geometry']]).set_index('Taz_num')['commuting']/100

    forecast['pop_emp_employed_out_of_jtmt_area']=forecast['pop_emp_employed']*forecast['commuting']

    emp_from_jtmt_area=forecast['pop_emp_employed'].sum().item()-forecast['pop_emp_employed_out_of_jtmt_area'].sum().item()

    emp_in_jtmt_area=emp_from_jtmt_area*1.06 #יוממות פנימה מחוץ למרחב

    emp_in_jtmt_area_without_mobile=emp_in_jtmt_area*0.94 #הפחחת עובדים ניידים
        
    ## יצירת תעסוקה לא עוקב
    add_emp_not_okev=emp_in_jtmt_area_without_mobile-emp_Education-emp_okev-emp_not_okev_current
    
    add_emp_not_okev_kibolet=EMP_kibolet['F{}'.format(y)].sum().item()-emp_not_okev_current
        
    add_emp_not_okev_filling=add_emp_not_okev/add_emp_not_okev_kibolet
    
    EMP_kibolet['emp_current']=(EMP_kibolet['F{}'.format(y)]-EMP_kibolet['emp_current'])*add_emp_not_okev_filling+EMP_kibolet['emp_current']
    
    emp_not_okev_current=EMP_kibolet['emp_current'].sum().item()
    
    EMP_kibolet.to_excel(r'{}\Intermediates\EMP_kibolet_{}_for_review_{}.xlsx'.format(software_data_folder_location,y,file_date))
    
    col=['Taz_num','ID','prec_from_id']

    df=EMP_kibolet_by_taz.reset_index()[col].merge(EMP_kibolet,on='ID',how='left')

    df['emp_current']=df['emp_current']*df['prec_from_id']

    forecast['emp_not_okev']=df.drop(columns=['geometry', 'geometry_buff']).pivot_table(index='Taz_num',aggfunc=sum)[['emp_current']]
    
    forecast=forecast.fillna(0)

    forecast['total_emp']=forecast['emp_not_okev']+forecast['emp_okev']+forecast['emp_Education']
    
    #הסיבה להפרדה כי המקדם השתתפות בעבודה באזורים לא חרדיים גבוהה מידי בשביל התלמידי ישיבה
    forecast.loc[forecast['main_secto']!="U_Orthodox",'pop']=forecast['pop']+forecast['yeshiva_dorms_pop_sum']

    forecast.loc[forecast['main_secto']!="U_Orthodox",'pop_15']=forecast['pop_15']+forecast['yeshiva_dorms_pop_15']

    forecast.loc[forecast['main_secto']!="U_Orthodox",'pop_20']=forecast['pop_20']+forecast['yeshiva_dorms_pop_20']
    
    forecast.loc[forecast['main_secto']!="U_Orthodox",'pop_25']=forecast['pop_25']+forecast['yeshiva_dorms_pop_25']
    
    col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

    forecast=forecast.join(emp_category_type[col]).fillna(0)

    for i in col:
        forecast['{}'.format(i)]=forecast['{}'.format(i)]*(forecast['emp_not_okev']+forecast['emp_okev'])
        
    forecast['yosh']=0

    forecast.loc[forecast['zonetype']=='Judea and Samaria','yosh']=1

    forecast['jerusalem_city']=0

    forecast.loc[forecast['in_jerusal']=='yes','jerusalem_city']=1

    forecast=forecast.reset_index()
    
    col=['Taz_num',
     'Taz_name',
     'Muni_Heb',
     'jeru_metro',
     'zonetype',
     'in_jerusal',
     'main_secto','jew',
     'student_dorms',
     'student_yeshiva',
     'student',
     'uni_students','emp_from_uni_student','emp_from_Yeshiva_student','yeshiva_dorms_pop_15',
     'yeshiva_dorms_pop_20',
     'yeshiva_dorms_pop_25',
     'yeshiva_dorms_pop_sum']
    
    forecast_current=forecast[col]

    ## ייצוא תחזית בפורמט
    col=['Taz_num',
    'yosh',
    'jeru_metro',
    'jerusalem_city',
    'main_secto',
    'hh',
    'pop',
    'pop_0',
    'pop_5',
    'pop_10',
    'pop_15',
    'pop_20',
    'pop_25',
    'pop_30',
    'pop_35',
    'pop_40',
    'pop_45',
    'pop_50',
    'pop_55',
    'pop_60',
    'pop_65',
    'pop_70',
    'pop_75up',
    'total_emp',
    'Indus',
    'Com_hotel',
    'Business',
    'Public',
    'emp_Education',
    'agri',
    'student',
    'uni_students',
    'student_yeshiva_and_kollim',
    'pop_emp_employed']

    col_new_name=['TAZ',
    'yosh',
    'in_jerusalem_metropolin',
    'jerusalem_city',
    'sector',
    'hh_total',
    'pop',
    'age0_4',
    'age5_9',
    'age10_14',
    'age15_19',
    'age20_24',
    'age25_29',
    'age30_34',
    'age35_39',
    'age40_44',
    'age45_49',
    'age50_54',
    'age55_59',
    'age60_64',
    'age65_69',
    'age70_74',
    'age75up',
    'emp_tot',
    'indus',
    'com_hotel',
    'business',
    'public',
    'education',
    'agri',
    'student',
    'univ',
    'UO_Hi_Ed',
    'pop_emp_employed']

    forecast_for_model=forecast[col]

    forecast_for_model.columns=col_new_name
    
    save_file_path=r'{}\Intermediates'.format(software_data_folder_location)
    
    save_excel_path=r'{}\{}_forecast_{}.xlsx'.format(save_file_path,file_date,y)

    forecast_for_model.to_excel(save_excel_path,index=False)
    
    logic_test_for_forecast(forecast_for_model)
    
    save_excel_path=r'{}\{}_forecast_{}_full.xlsx'.format(save_file_path,file_date,y)

    forecast.to_excel(save_excel_path,index=False)

C:\Users\dpere\AppData\Local\Temp\ipykernel_6232\2396379921.py:9: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  demad_for_students_by_muni_and_sector=forecast.pivot_table(index=('Muni_Heb','jew'),aggfunc=sum)[['student_demand','student']]
C:\Users\dpere\AppData\Local\Temp\ipykernel_6232\2396379921.py:91: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2.55572217e+02 2.92906508e+02 2.43943596e+02 1.48712286e+02
 2.07869889e+02 5.85309175e+01 4.60465237e+01 9.40979845e+01
 1.21735834e+02 1.81574563e+02 3.46345373e+01 1.76231431e+02
 3.26002242e+02 4.11835341e+02 6.62788552e+01 1.84883523e+02
 7.28784663e+01 1.05359874e+02 2.02346042e+02 2.51311893e+02
 1.77313189e+02 3.11911071e-01 1.93027053e+02 1.95570552e+02
 1.01396154e+02 5.50

taz num under 0: []
taz num duplicate: []
yosh_unique: [0, 1]
in_jerusalem_metropolin_unique: [1, 0]
jerusalem_city_unique: [1, 0]
sector_unique: ['arabs_behined_seperation_wall', 'Arab', 'U_Orthodox', 'Jewish', 'Palestinian']
taz num with minus: []
pop more then hh: []
hh missing pop: []
pop missing hh: []
pop vs age dis Mistake: []
taz num with age dis prob: []
taz num with emp total prob: []
taz num with emp dis prob: []
taz num with pop_emp_employed worng : []


C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz[col].sum(axis=1)-taz['pop']
C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz['pop_check'].round(0)
C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:196: SettingWithCopyWarning: 
A value is t

taz num under 0: []
taz num duplicate: []
yosh_unique: [0, 1]
in_jerusalem_metropolin_unique: [1, 0]
jerusalem_city_unique: [1, 0]
sector_unique: ['arabs_behined_seperation_wall', 'Arab', 'U_Orthodox', 'Jewish', 'Palestinian']
taz num with minus: []
pop more then hh: []
hh missing pop: []
pop missing hh: []
pop vs age dis Mistake: []
taz num with age dis prob: []
taz num with emp total prob: []
taz num with emp dis prob: []
taz num with pop_emp_employed worng : []


C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz[col].sum(axis=1)-taz['pop']
C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz['pop_check'].round(0)
C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:196: SettingWithCopyWarning: 
A value is t

...

C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz[col].sum(axis=1)-taz['pop']
C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz['pop_check'].round(0)
C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:196: SettingWithCopyWarning: 
A value is t

taz num under 0: []
taz num duplicate: []
yosh_unique: [0, 1]
in_jerusalem_metropolin_unique: [1, 0]
jerusalem_city_unique: [1, 0]
sector_unique: ['arabs_behined_seperation_wall', 'Arab', 'U_Orthodox', 'Jewish', 'Palestinian']
taz num with minus: []
pop more then hh: []
hh missing pop: []
pop missing hh: []
pop vs age dis Mistake: []
taz num with age dis prob: []
taz num with emp total prob: []
taz num with emp dis prob: []
taz num with pop_emp_employed worng : []


C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz[col].sum(axis=1)-taz['pop']
C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz['pop_check'].round(0)
C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:196: SettingWithCopyWarning: 
A value is t

taz num under 0: []
taz num duplicate: []
yosh_unique: [0, 1]
in_jerusalem_metropolin_unique: [1, 0]
jerusalem_city_unique: [1, 0]
sector_unique: ['arabs_behined_seperation_wall', 'Arab', 'U_Orthodox', 'Jewish', 'Palestinian']
taz num with minus: []
pop more then hh: []
hh missing pop: []
pop missing hh: []
pop vs age dis Mistake: []
taz num with age dis prob: []
taz num with emp total prob: []
taz num with emp dis prob: []
taz num with pop_emp_employed worng : []


C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz[col].sum(axis=1)-taz['pop']
C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz['pop_check'].round(0)
C:\Users\dpere\Documents\JTMT\forecast_git\create_forecast_basic\future\global_functions.py:196: SettingWithCopyWarning: 
A value is t

## תוצאות